In [1]:
#%pip install pyspark

1. AQI is a non-linear indication of how a specific airborne chemical affects public health. Calculate the AQI values for each indicator's daily mean (see https://www.airnow.gov/sites/default/files/2020-05/aqi-technical-assistance-document-sept2018.pdf page 14 - 15, use the longest periods available; 8 or 24 hrs). Note that this might not correspond perfectly with the dataset's existing AQI values as it depends on the timescale used (1hr, 8hrs or other). For the rest of the questions, you are allowed to use the dataset's AQI values.

In [2]:
#create SparkContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import operator

conf = SparkConf().setAppName("test").setMaster("local")
sc = SparkContext(conf=conf)

In [3]:
#Creating spark session and loading data
spark = SparkSession(sc)
pollutiondf = spark.read.format("csv").option("header", "true").load("pollution_2000_2021.csv")
pollutiondf.show(10)

+----------+----+-----+---+--------------------+-------+--------+-------+--------+----------------+---------------+------+------------------+----------------+---------------+------+------------------+-----------------+----------------+-------+---------+-----------------+----------------+-------+
|      Date|Year|Month|Day|             Address|  State|  County|   City| O3 Mean|O3 1st Max Value|O3 1st Max Hour|O3 AQI|           CO Mean|CO 1st Max Value|CO 1st Max Hour|CO AQI|          SO2 Mean|SO2 1st Max Value|SO2 1st Max Hour|SO2 AQI| NO2 Mean|NO2 1st Max Value|NO2 1st Max Hour|NO2 AQI|
+----------+----+-----+---+--------------------+-------+--------+-------+--------+----------------+---------------+------+------------------+----------------+---------------+------+------------------+-----------------+----------------+-------+---------+-----------------+----------------+-------+
|2000-01-01|2000|    1|  1|1645 E ROOSEVELT ...|Arizona|Maricopa|Phoenix|0.019765|            0.04|          

In [4]:
# Define Breakpoints for AQI Calculation
# Format: (C_low, C_high, I_low, I_high)
O3_BREAKPOINTS = [
    (0.000, 0.054, 0, 50),
    (0.055, 0.070, 51, 100),
    (0.071, 0.085, 101, 150),
    (0.086, 0.105, 151, 200),
    (0.106, 0.200, 201, 300),
]
CO_BREAKPOINTS = [
    (0.0, 4.4, 0, 50),
    (4.5, 9.4, 51, 100),
    (9.5, 12.4, 101, 150),
    (12.5, 15.4, 151, 200),
    (15.5, 30.4, 201, 300),
    (30.5, 40.4, 301, 400),
    (40.5, 50.4, 401, 500),
]
SO2_BREAKPOINTS = [
    (0, 35, 0, 50),
    (36, 75, 51, 100),
    (76, 185, 101, 150),
    (186, 304, 151, 200),
    (305, 604, 201, 300),
    (605, 804, 301, 400),
    (805, 1004, 401, 500),
]
NO2_BREAKPOINTS = [
    (0, 53, 0, 50),
    (54, 100, 51, 100),
    (101, 360, 101, 150),
    (361, 649, 151, 200),
    (650, 1249, 201, 300),
    (1250, 1649, 301, 400),
    (1650, 2049, 401, 500),
]

In [5]:
# AQI Calculation Function
#Calculate AQI for a given pollutant based on predefined breakpoints and add it as a new column to the DataFrame.
def calculate_aqi(df, pollutant, breakpoints):
    aqi_expr = None
    for C_low, C_high, I_low, I_high in breakpoints:
        # define condition and AQI formula for each breakpoint
        condition = (F.col(pollutant) >= C_low) & (F.col(pollutant) <= C_high)
        aqi_formula = ((I_high - I_low) / (C_high - C_low)) * (F.col(pollutant) - C_low) + I_low
        if aqi_expr is None:
            # create the AQI expression for the first condition
            aqi_expr = F.when(condition, aqi_formula)
        else:
            # expand the AQI expression with the new condition
            aqi_expr = aqi_expr.when(condition, aqi_formula)
    
    return df.withColumn(f"{pollutant}_AQI", aqi_expr)

In [6]:
pollutiondf.select("Date", "O3 Mean","O3 AQI","CO Mean","CO AQI","SO2 Mean","SO2 AQI","NO2 Mean","NO2 AQI")
# Calculate AQI for each pollutant
pollutiondf = calculate_aqi(pollutiondf, "O3 Mean", O3_BREAKPOINTS)
pollutiondf = calculate_aqi(pollutiondf, "CO Mean", CO_BREAKPOINTS)
pollutiondf = calculate_aqi(pollutiondf, "SO2 Mean", SO2_BREAKPOINTS)
pollutiondf = calculate_aqi(pollutiondf, "NO2 Mean", NO2_BREAKPOINTS)

# Show results
pollutiondf.select("Date", "O3 Mean", "O3 AQI", "O3 Mean_AQI", "CO Mean", "CO AQI", "CO Mean_AQI", "SO2 Mean", "SO2 AQI", "SO2 Mean_AQI", "NO2 Mean", "NO2 AQI", "NO2 Mean_AQI").show(10)

+----------+--------+------+------------------+------------------+------+------------------+------------------+-------+------------------+---------+-------+------------------+
|      Date| O3 Mean|O3 AQI|       O3 Mean_AQI|           CO Mean|CO AQI|       CO Mean_AQI|          SO2 Mean|SO2 AQI|      SO2 Mean_AQI| NO2 Mean|NO2 AQI|      NO2 Mean_AQI|
+----------+--------+------+------------------+------------------+------+------------------+------------------+-------+------------------+---------+-------+------------------+
|2000-01-01|0.019765|    37|18.300925925925927|0.8789469999999999|  25.0| 9.988034090909089|               3.0|   13.0| 4.285714285714286|19.041667|     46| 17.96383679245283|
|2000-01-02|0.015882|    30|14.705555555555557|          1.066667|  26.0|12.121215909090909|          1.958333|    4.0|2.7976185714285715|22.958333|     34| 21.65880471698113|
|2000-01-03|0.009353|    15| 8.660185185185187|            1.7625|  28.0| 20.02840909090909|              5.25|   16.0| 

2. Calculate the all-time most polluted cities for each of the indicators (use the daily "x Mean" as values, not the AQI for a specific date). Show the top 10 for each. You can show the results in separate tables, but write your code efficiently!

In [7]:
#2. Calculate the all-time most polluted cities for each of the indicators (use the daily "x Mean" as values, not the AQI for a specific date). Show the top 10 for each. You can show the results in separate tables, but write your code efficiently!
# Calculate the average of each pollutant for each city
average_pollution = pollutiondf.groupBy("City").agg(
    F.avg("O3 Mean").alias("O3 Mean"),
    F.avg("CO Mean").alias("CO Mean"),
    F.avg("SO2 Mean").alias("SO2 Mean"),
    F.avg("NO2 Mean").alias("NO2 Mean")
)
top_10_O3 = average_pollution.orderBy(F.desc("O3 Mean")).limit(10)
top_10_CO = average_pollution.orderBy(F.desc("CO Mean")).limit(10)
top_10_SO2 = average_pollution.orderBy(F.desc("SO2 Mean")).limit(10)
top_10_NO2 = average_pollution.orderBy(F.desc("NO2 Mean")).limit(10)

In [8]:
# Top 10 most polluted cities O3
top_10_O3.select("City", "O3 Mean").show()

+--------------------+--------------------+
|                City|             O3 Mean|
+--------------------+--------------------+
|           Roosevelt|0.049524846153846155|
|        Boulder City| 0.04127796198347112|
|             Capitan|0.040639070016793566|
|          Ponca City|0.040218525101763894|
|       Winston-Salem|0.038678721649484514|
|         Victorville| 0.03717309678259743|
|Vandenberg Air Fo...|0.037122593284143884|
|            Cornwall|0.036468483351235315|
|             Hampton|0.035784318104184455|
|         Cherry Tree| 0.03562345342312008|
+--------------------+--------------------+



In [9]:
# Top 10 most polluted cities CO
top_10_CO.select("City", "CO Mean").show()

+-------------+------------------+
|         City|           CO Mean|
+-------------+------------------+
|    Hawthorne|0.8651948047650568|
|  Bakersfield|0.8339144722222221|
|    San Diego|0.8267505104529564|
|Seven Corners|0.8043782304250554|
|      Burbank| 0.750850411317565|
|       Kenner|0.6993138676470587|
|  Chula Vista|0.6926273021978063|
|   Washington|0.6423517303899974|
|      Chicago|0.6349125815602833|
|     Calexico|0.6326138121660547|
+-------------+------------------+



In [10]:
# Top 10 most polluted cities SO2
top_10_SO2.select("City", "SO2 Mean").show()

+--------------------+------------------+
|                City|          SO2 Mean|
+--------------------+------------------+
|       Seven Corners|11.300477624161081|
|          New Castle| 6.471878813880126|
|          Greensburg| 6.368438866987191|
|              McLean|6.3591178760731655|
|             Reading|  6.16206293768997|
|        Beaver Falls| 5.774642403217163|
|             St. Ann| 5.719362558139535|
|           Fairbanks| 5.634989615435359|
|Calumet City (PU ...| 5.470537862992126|
|           Henderson| 5.228105508379882|
+--------------------+------------------+



In [11]:
# Top 10 most polluted cities NO2
top_10_NO2.select("City", "NO2 Mean").show()

+--------------------+------------------+
|                City|          NO2 Mean|
+--------------------+------------------+
|             Chicago|31.451528046099323|
|             Burbank|30.431701535472882|
|         Bakersfield| 29.02237005555556|
|           Hawthorne|24.985028121111856|
|    West Los Angeles|23.778210487455194|
|          Scottsdale| 22.61990255223194|
|          Long Beach|22.071382071279952|
|Calumet City (PU ...|21.777968725984273|
|              Cicero|21.262819596439172|
|               Ladue| 21.09329080184332|
+--------------------+------------------+



3. Find the statistical correlation (if any!) between traffic-based AQI indicators (SO2/NO2). Note that either may also be a result of industrial activity, so the correlation may be far from perfect. Additionally, determine if the climate difference between northern and southern states has an influence on CO and O3 AQI (pick one northern and one southern state, you don't have to do all of them). You are free to figure out your own approach how to do this, but document it well. Hint: don't start with either Alaska or Hawaii, they're statistical outliers.

In [12]:
#3. Find the statistical correlation (if any!) between traffic-based AQI indicators (SO2/NO2). Note that either may also be a result of industrial activity, so the correlation may be far from perfect. Additionally, determine if the climate difference between northern and southern states has an influence on CO and O3 AQI (pick one northern and one southern state, you don't have to do all of them). You are free to figure out your own approach how to do this, but document it well. Hint: don't start with either Alaska or Hawaii, they're statistical outliers.
# Convert SO2 AQI and NO2 AQI columns to DoubleType
pollutiondf = pollutiondf.withColumn("SO2 AQI", F.col("SO2 AQI").cast(DoubleType()))
pollutiondf = pollutiondf.withColumn("NO2 AQI", F.col("NO2 AQI").cast(DoubleType()))

# Correlation between SO2 and NO2
# Functioon to interpret the correlation value
def interpret_correlation(value):
    if abs(value) < 0.1:
        print("No or very weak correlation")
    elif abs(value) < 0.3:
        print("Weak correlation")
    elif abs(value) < 0.5:
        print("Moderate correlation")
    elif abs(value) < 0.9:
        print("Strong correlation")
    else:
        print("Very strong correlation")
correlation = pollutiondf.stat.corr("SO2 AQI", "NO2 AQI")
print(f"Correlation between SO2 Mean AQI and NO2 Mean AQI: {correlation}")
interpret_correlation(correlation)


Correlation between SO2 Mean AQI and NO2 Mean AQI: 0.30259658257408323
Moderate correlation


In [13]:
# Influence of climate on CO and O3 AQI
# Select one northern and one southern state
northern_state="Minnesota"
southern_state="Texas"
# Filter data for the selected states
northern_data = pollutiondf.filter(F.col("State") == northern_state)
southern_data = pollutiondf.filter(F.col("State") == southern_state)

# Calculate the average AQI for CO and O3 for each state per month
northern_data = northern_data.groupBy("State", "Month").agg(
    F.avg("CO Mean_AQI").alias("CO Mean_AQI"),
    F.avg("O3 Mean_AQI").alias("O3 Mean_AQI")
)
southern_data = southern_data.groupBy("State", "Month").agg(
    F.avg("CO Mean_AQI").alias("CO Mean_AQI"),
    F.avg("O3 Mean_AQI").alias("O3 Mean_AQI")
)
# Ensure the Month column is cast to an integer
northern_data = northern_data.withColumn("Month", F.col("Month").cast("int"))
southern_data = southern_data.withColumn("Month", F.col("Month").cast("int"))

# Order by Month correctly
northern_data.orderBy("Month").show()
southern_data.orderBy("Month").show()

+---------+-----+------------------+------------------+
|    State|Month|       CO Mean_AQI|       O3 Mean_AQI|
+---------+-----+------------------+------------------+
|Minnesota|    1|3.0387538086913075| 20.94612332112332|
|Minnesota|    2| 2.879849328780026|26.488596238725787|
|Minnesota|    3|2.7621064049586774|30.455564908342676|
|Minnesota|    4|2.6189013082505728| 33.37626828509182|
|Minnesota|    5|2.4239084022038555| 33.23841561648229|
|Minnesota|    6|2.3877163352272706| 34.10202185662057|
|Minnesota|    7|2.7191370172968607|  28.2870411340544|
|Minnesota|    8| 2.864411054994387|24.282102575826862|
|Minnesota|    9|2.6479837403288196|21.781176122931424|
|Minnesota|   10|3.0184358483086653| 19.92481589776911|
|Minnesota|   11|3.3595823484848495|17.289456790123445|
|Minnesota|   12|2.9627889185977416|17.336147421931734|
+---------+-----+------------------+------------------+

+-----+-----+------------------+------------------+
|State|Month|       CO Mean_AQI|       O3 Mean_AQI|

In [14]:
# Calculate average per year
northern_data = northern_data.groupBy("State").agg(
    F.avg("CO Mean_AQI").alias("CO Mean_AQI"),
    F.avg("O3 Mean_AQI").alias("O3 Mean_AQI")
)
southern_data = southern_data.groupBy("State").agg(
    F.avg("CO Mean_AQI").alias("CO Mean_AQI"),
    F.avg("O3 Mean_AQI").alias("O3 Mean_AQI")
)
northern_data.show()
southern_data.show()


+---------+-----------------+-----------------+
|    State|      CO Mean_AQI|      O3 Mean_AQI|
+---------+-----------------+-----------------+
|Minnesota|2.806964543010253|25.62564418075195|
+---------+-----------------+-----------------+

+-----+-----------------+-----------------+
|State|      CO Mean_AQI|      O3 Mean_AQI|
+-----+-----------------+-----------------+
|Texas|2.756071629532524|25.89437391140008|
+-----+-----------------+-----------------+



The output of the two cells above show that there is little difference
between the CO and O3 AQI values for the northern and southern states.
The difference is not significant enough to conclude that the climate has an influence on the AQI values.

4. For each city, calculate if traffic-based indicators, heating-based indicators (CO in this case), or weather-bound indicators (O3) are the major polluting factor and add this information in a separate column "MajorFactor" (values "traffic", "heating", "weather").

In [15]:
# traffic-based indicators -> NO2 + S02
# Identify the major polution indicator for each city in the dataset
# calculate the average pollution levels per city for different indicators:
# - Traffic-related pollution: NO2 and SO2
# - Heating-related pollution: CO
# - Weather-influenced pollution: O3
pollution_avg = pollutiondf.groupBy("City").agg(
    F.avg("NO2 Mean").alias("Avg_NO2"),  
    F.avg("SO2 Mean").alias("Avg_SO2"),  
    F.avg("CO Mean").alias("Avg_CO"),    
    F.avg("O3 Mean").alias("Avg_O3")  )   

# Determine the major pollution factor for each city
pollution_avg = pollution_avg.withColumn(
    "MajorFactor",
    F.when((F.col("Avg_NO2") + F.col("Avg_SO2") > F.col("Avg_CO")) & (F.col("Avg_NO2") + F.col("Avg_SO2") > F.col("Avg_O3")), "traffic")
    .when((F.col("Avg_CO") > F.col("Avg_NO2") + F.col("Avg_SO2")) & (F.col("Avg_CO") > F.col("Avg_O3")), "heating")
    .otherwise("weather")
)

# example of a few cities (column 1) with their major polluting factir in the second column.
# example of a few cities with their dominant pollution source
pollution_avg.select("City", "MajorFactor").show()

+--------------------+-----------+
|                City|MajorFactor|
+--------------------+-----------+
|           Fairbanks|    traffic|
|             Phoenix|    traffic|
|           Pittsburg|    traffic|
|       Not in a city|    traffic|
|         Grantsville|    traffic|
|            Rubidoux|    traffic|
|  Breckenridge Hills|    traffic|
|Indianapolis (Rem...|    traffic|
|  East Highland Park|    traffic|
|              Dallas|    traffic|
|Lexington-Fayette...|    traffic|
|             Oakland|    traffic|
|          Manchester|    traffic|
|           Cupertino|    traffic|
|          Scottsdale|    traffic|
|             Fontana|    traffic|
|             Rutland|    traffic|
|               Welby|    traffic|
|         Bakersfield|    traffic|
|          Holtsville|    traffic|
+--------------------+-----------+
only showing top 20 rows



Table Showing the cities int the dataset with their MajorFactor.

5. Find the city with the greatest pollution spree, calculated as counting the number of consecutive days where the AQI for a chosen indicator is higher than 50. Extend your solution so you can efficiently calculate it for any AQI.

In [16]:
# Find the city with the greatest pollution spree of high pollution days
# We define a pollution streak as consecutive days where the AQI is higher than 50.

# To calculate it for another AQI you only need to change the following line "03 AQI" becomes "S02/NO2/CO AQI"
chosen_aqi = "O3 AQI"
# if higher than 50.  
threshold = 50  

# Create a column that marks whether AQI is above the threshold (in this case 50)
pollutiondf = pollutiondf.withColumn("Above_Threshold", (F.col(chosen_aqi) > threshold).cast("int"))

# Define a window specification partitioned by city and ordered by date
windowSpec = Window.partitionBy("City").orderBy("Date")

# Identify when a new streak (consecutive days) starts (i.e., the first day AQI exceeds 50 after being below it)
pollutiondf = pollutiondf.withColumn("StreakStart", F.when((F.col("Above_Threshold") == 1) & (F.lag("Above_Threshold", 1).over(windowSpec) == 0), 1).otherwise(0))
# (F.lag("Above_Threshold", 1).over(windowSpec) == 0), 1) -> checks if previous day exceeds the AQI-threshold.

# sum to group consecutive high-AQI days into streaks, count the length of each streak, at last search the longest sprees per city.
pollutiondf = pollutiondf.withColumn("StreakGroup", F.sum("StreakStart").over(windowSpec))
streaks = pollutiondf.filter(F.col("Above_Threshold") == 1).groupBy("City", "StreakGroup").agg(F.count("Above_Threshold").alias("StreakLength"))
longest_streak = streaks.groupBy("City").agg(F.max("StreakLength").alias("MaxStreak"))

# example of a few cities with the greatest pollution sprees
most_polluted_city = longest_streak.orderBy(F.desc("MaxStreak")).limit(5)
most_polluted_city.show()


+--------------+---------+
|          City|MaxStreak|
+--------------+---------+
|      Rubidoux|      332|
|Salt Lake City|       85|
|   Victorville|       74|
|        Fresno|       73|
|       Fontana|       69|
+--------------+---------+



The city with the greatest pollution spree is Rubidoux with 332 consecutive days, where the 03-AQI is higher than 50.